### Imports

In [1]:
import sys

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
from matplotlib.ticker import ScalarFormatter

from astropy import units as u
from astropy import constants as const
from astropy.table import QTable

from pathlib import Path
import os

sys.path.insert(0, '/home/emelie/Desktop/Codes/PETAR/PeTar-master/tools/')
from analysis import *

In [2]:
plt.rcParams.update({'xtick.labelsize':13, 'ytick.labelsize':13, 'axes.titlesize':16, 
                     'axes.grid':True, 'axes.labelsize':15, 'legend.fontsize':13})

### Mahalanobis distance

The distance in 6D phace space is given by the Mahalanobis distance: 

\begin{equation}
    d_{M} (\mathbf{x_1}, \mathbf{x_2}:C) = \sqrt((\mathbf{x_1} - \mathbf{x_2})C^{-1}(\mathbf{x_1} - \mathbf{x_2})^T)
\end{equation}

where $\mathbf{x_1}$ and$\mathbf{x_2}$ are two points in 6D phace space and $C$ is given by a $6 \times 6$ matrix with elements following:

\begin{equation}
    C_{ij} =  \langle(x_i - <x_i>) (x_j - <x_j>) \rangle,
\end{equation}

where $\langle \rangle$ represent the mean value. This gives a matrix like:

\begin{equation}
    C =  \begin{pmatrix}
            C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
            C_{21} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
            C_{31} & C_{32} & C_{33} & C_{34} & C_{35} & C_{36} \\
            C_{41} & C_{42} & C_{43} & C_{44} & C_{45} & C_{46} \\
            C_{51} & C_{52} & C_{53} & C_{54} & C_{55} & C_{56} \\
            C_{61} & C_{62} & C_{63} & C_{64} & C_{65} & C_{66} \\
         \end{pmatrix}.
\end{equation}


I have $d_{nj} C^{-1}_{ji}d^{T}_{in}$ and want to sum such that it has shape (n). 

In [ ]:
def mahalanobis_distance(first_pnt, space_6d):
    """
    Summary: 
    Calculates the Mahalanobis distance between one point (first_point) and other points closeby (space_6d)
    ----------------------------------------------------------
    Parameters:
    -----------
    first_pnt: 1d array
        Array with 6d phase space data (x, y, z, vx, vy, vz) for first point 
    
    space_6d: 2d-array
        Array with 6d phase space data (x, y, z, vx, vy, vz) for closeby points
        
    Returns:
    --------
    mahalanobis_dist: 1d array
        Mahalanobis distances between each closeby point and the first point
        
    """
    
    # Calculate the difference vector 
    diff = first_pnt - space_6d
    
    covariance_matrix = np.cov(space_6d, rowvar=False)
    inv_covariance_matrix = np.linalg.inv(covariance_matrix)
    
    
    # Calculate the Mahalanobis distance
    mahalanobis_dist = np.sqrt(np.einsum('nj,ji,in->n', diff, inv_covariance_matrix, diff.T))

    return mahalanobis_dist

### Testing Mahalanobis

In [ ]:
space_data_6d = run_fid_large_mod_data[:, 1:7, -1]
max_z_pos = np.where(space_data_6d[:, 2]==np.max(space_data_6d[:, 2]))[0]

first_point = space_data_6d[0, :]
space_6d_data2 = space_data_6d
print(np.shape(space_6d_data2))
space_6d_data2 = np.delete(space_6d_data2, max_z_pos, axis=0)
print(np.shape(space_6d_data2))


data1 = space_6d_data2[0:100000, :]
data2 = space_6d_data2[100001:200000, :]
data3 = space_6d_data2[200001:300000, :]
data4 = space_6d_data2[300001:400000, :]
data5 = space_6d_data2[400001:500000, :]
data6 = space_6d_data2[500001:600000, :]
data7 = space_6d_data2[600001:700000, :]
data8 = space_6d_data2[700001:800000, :]
data9 = space_6d_data2[800001:900000, :]
data10 = space_6d_data2[900001:-1, :]


datas = [data1, data2, data3, data4, data5, data6, data7, data8, data9, data10]


beg = np.array([0, 100001, 200001, 300001, 400001, 500001, 600001, 700001, 800001, 900001])
end = np.array([100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, -1])

In [ ]:
dms = np.zeros((len(space_6d_data2)))

for i, data in enumerate(datas):
    dm = mahalanobis_distance(first_point, data)
    dms[beg[i]:end[i]] = dm

    
print(np.shape(dms))
print(dms)

### Plotting Mahalanobis distances from first point to the rest of the dataset

In [ ]:
fig3, ax3 = plt.subplots(figsize=(11, 10))


sc = ax3.scatter(space_6d_data2[:,0,], space_6d_data2[:,2,], c=dms, s=1, 
                 cmap='bwr')
plt.colorbar(sc, label='6D distance')
ax3.set_xlim(xmin=-0.2e5, xmax=0.7e5) #xmin=-0.7e5, xmax=0.8e5
ax3.set_ylim(ymin=-0.85e5)
ax3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3.yaxis.get_major_formatter().set_powerlimits((0, 0))

#ax3.set_title('')
ax3.set_xlabel('x [pc]')
ax3.set_ylabel('z [pc]')

plt.show()

### Stream walk function

In [ ]:
def stream_walk(space_6d, max_city_dist, max_block_dist):
    """
    Summary:
    ---------
    City means a larger amount of particles that are closeby, the particles in the city are the citizens. 
    Defining this makes other calculations much faster as it limits the amount of particles. 
    Citizens are found by x, y, z coordinates and radial distance. 
    The block contains the closest particles, the neighbours. I choose the next particle from this category.
    Neighbours are foudn through the Mahalanobis distance. 
    ...
    ----------------------------------------------------------
    Parameters:
    -----------
    space_6d: 2d-array
        Array with space coordinates with columns x, y, z, vx, vy, vz
        
    max_city_dist: float or int
        Maximum distance to particles that are closeby to save computation time
        
    max_block_dist: float or int
        Maximum distance to particles that are neighbours to the particle
        
    Return:
    --------
    path: 2d array
        Array with points along the path of the stream
    """
    
    space_6d_copy = space_6d.copy() # copying space data to keep input array intact
    #print(np.shape(space_6d_copy))
    
    # FIRST PARTICLE
    #-------------------------------------------------------------------------------------------
    # Finding first particle position, defining it as z_max
    first_particle_pos = np.where(space_6d[:, 2]==np.max(space_6d[:, 2]))[0][0] 
    # Extracting first particle information
    first_particle = space_6d_copy[first_particle_pos, :] 
    # Creating path array with first particle at the beginning
    path = np.array([first_particle]) 
    # Removing first particle from list
    space_6d_copy = np.delete(space_6d_copy, first_particle_pos, axis=0)
    #print(np.shape(space_6d_copy))
    
    
    # SECOND PARTICLE: need two first particles to initiate loop
    #-------------------------------------------------------------------------------------------
    # Extracting first particle data
    previous_particle = path[-1, :]
    
    ## Finding city particles ##
    # 3D distances - difference
    difference = previous_particle[np.newaxis, :] - space_6d_copy
    # 3D distances - r = sqrt(x² + y² + z²)
    distance_3d = np.sqrt(difference[:, 0]**2 + difference[:, 1]**2 + difference[:, 2]**2)
    # Finding the particles in the city
    city_mask = distance_3d<=max_city_dist
    # Extracting the citizen data
    citizens = space_6d_copy[city_mask]
    print(f'c1 = {len(citizens)}')
    # Extracting the positions of the citizens in the whole data set 
    citizens_positions = np.where(distance_3d<=max_city_dist)[0]
    # Mahalanobis distances to citizens
    dist_m = mahalanobis_distance(previous_particle, citizens)
    # Fincing closest citizen
    min_distance = np.min(dist_m)
    # Position of closest citizen
    min_distance_pos = np.where(dist_m==min_distance)[0]
    
    # Extracting second particle data
    second_particle = citizens[min_distance_pos, :]
    # Adding to path
    path = np.vstack([path, second_particle])
    # Remove from particle list
    space_6d_copy = np.delete(space_6d_copy, min_distance_pos, axis=0) 
    
    # SETTING UP LOOP CONDITION
    #--------------------------------------------------------------------------------------------
    # Previous particle, here second
    previous_particle = path[-1, :]
    # 3D distances -> difference
    difference = previous_particle[np.newaxis, :] - space_6d_copy
    # 3D distances -> r = sqrt(x² + y² + z²)
    distance_3d = np.sqrt(difference[:, 0]**2 + difference[:, 1]**2 + difference[:, 2]**2)
    print(np.min(distance_3d), np.max(distance_3d))
    # Finding the particles in the city
    city_mask = distance_3d<=max_city_dist
    # Extracting the citizen data
    citizens = space_6d_copy[city_mask]
    # Extracting the positions of the citizens in the whole data set 
    citizens_positions = np.where(distance_3d<=max_city_dist)[0]
    print(f'c2 = {len(citizens)}')
    
    # Mahalanobis distances to citizens
    dist_m = mahalanobis_distance(previous_particle, citizens)
    print(np.min(dist_m), np.max(dist_m))
    
    
    # While there are particles closer within block distances !!!MIGHT HAVE TO CHANGE!!!
    while any(dist_m<=max_block_dist): 
        print('Starting loop')
        # Finding neighbours
        neighbours_mask = (dist_m<=max_block_dist)
        # Extracting neighbours
        neighbours = citizens[neighbours_mask, :] 
        # Mahalanobis distance from previous particle
        neighbours_dm = dist_m[neighbours_mask]
        # Finding neighbour's positions in city
        neighbours_city_positions = np.where(dist_m<=max_block_dist)[0]
        # Finding neighbour's positions in data
        neighbours_positions = citizens_positions[neighbours_city_positions]
        print(f'n = {len(neighbours)}')
        
        
        # If there are "few" neighbours
        if len(neighbours)<=1500:
            # Finding minimum distance to other neighbours
            min_distance = np.min(neighbours_dm) 
            # Finding where closest neighbour
            min_distance_pos = np.where(neighbours_dm==min_distance)[0] 
            # Data for closest neighbour
            closest_particle = neighbours[min_distance_pos, :]
            #closest_particle_pos = neighbours_positions[min_distance_pos]
            # Adding neighbour to stream path
            path = np.vstack([path, closest_particle])
            
            closest_neighbours_mask = neighbours_dm<=0.5*max_block_dist
            closest_neighbours_pos = neighbours_positions[closest_neighbours_mask]
            # Removing all neighbouring particles from list of particles
            space_6d_copy = np.delete(space_6d_copy, closest_neighbours_pos, axis=0)
        
        # If there are very many neighbours
        else:
            # Number of citizens for each neighbour
            densities = np.zeros(len(neighbours))
            volume = np.zeros((len(neighbours), len(space_6d_copy)))
            
            ### DOES NOT WORK! ###
            # Differences in 3D (x, y, z)
            diffs = neighbours[:, np.newaxis, :3] - space_6d_copy[:, :3]
            #print(np.shape(diffs)) # Shape: (N_n, N_all, 3D)
            # Calculating r
            dists_squared = np.sum(diffs**2, axis=2) # Need shape: (N_n, N_all)
            #print(np.shape(dists_squared))
            dists_av = np.sqrt(dists_squared)
            #print(np.min(dists_av), np.max(dists_av))
            # Finding city limits
            citizens_mask = dists_av <= max_city_dist
            non_citizens_mask = dists_av > max_city_dist
            # Finding closest particles
            volume[citizens_mask] = 1
            volume[non_citizens_mask] = 0
            # Calculating number of citizens for each neighbour
            densities = np.sum(volume, axis=1)
            #print(f'densities = {densities}')
            # Finding neighbour with most citizens
            max_dens = np.where(densities==np.max(densities))[0]
            #print(max_dens)
            
            # If there are more than one neighbour with maximum citizens
            if len(max_dens)>1: 
                # Neighbours with maximum citizens
                max_dens_neighbours = neighbours[max_dens, :]
                # Taking the first close neighbour with maximum citizens
                closest_particle = max_dens_neighbours[0, :]
                # Adding closest neighbour to stream path
                np.vstack([path, closest_particle]) 
            
            # If there is only one neighbour with maximum citizens
            else:
                # Neighbour in right direction
                closest_neighbour = neighbours[max_dens, :]
                # Adding closest neighbour to stream path
                path = np.vstack([path, closest_neighbour]) 
        
        
        
            # Removing all neighbouring particles from list of particles
            space_6d_copy = np.delete(space_6d_copy, neighbours_positions, axis=0)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        ax.scatter(space_6d_copy[:, 0], space_6d_copy[:, 2], c='b', alpha=0.5, s=1)
        ax.plot(path[:, 0], path[:, 2], color='r', marker='x')
        
        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.xaxis.get_major_formatter().set_powerlimits((0, 0))
        ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.yaxis.get_major_formatter().set_powerlimits((0, 0))

        ax.set_title('Stream walk path and stream data')
        ax.set_xlabel('x [pc]')
        ax.set_ylabel('z [pc]')
        plt.show()
        
        
        # Setting new distances for the loop
        previous_particle = path[-1, :]
        # 3D distances -> difference
        difference = previous_particle[np.newaxis, :] - space_6d_copy
        # 3D distances -> r = sqrt(x² + y² + z²)
        distance_3d = np.sqrt(difference[:, 0]**2 + difference[:, 1]**2 + difference[:, 2]**2)
        # Finding the particles in the city
        city_mask = distance_3d<=max_city_dist
        # Extracting the citizen data
        citizens = space_6d_copy[city_mask]
        # Extracting the positions of the citizens in the whole data set 
        citizens_positions = np.where(distance_3d<=max_city_dist)[0]
        print(f'c = {len(citizens)}')
        print()
    
        # Mahalanobis distances to citizens
        dist_m = mahalanobis_distance(previous_particle, citizens)
    
    return path

### Testing stream walk function

In [ ]:
space_data_6d = run_fid_large_mod_data[:, 1:7, -1]


stream_path = stream_walk(space_data_6d, max_city_dist=0.4e5, max_block_dist=5)
print(np.shape(stream_path))

In [ ]:
fig2, ax2 = plt.subplots(figsize=(9, 8))


sc = ax2.scatter(run_fid_large_mod_data[:,1,-1], run_fid_large_mod_data[:,3,-1], c='b', alpha=0.3, s=1)

plt.colorbar(sc, label='6D distance')

ax2.plot(stream_path[:, 0], stream_path[:, 2], color='r', marker='x')
#ax2.scatter(run_fid_large_mod_data[645881,1,-1], run_fid_large_mod_data[645881,3,-1], c='b')
#ax2.set_xlim(xmin=-0.7e5, xmax=0.8e5) #xmin=-0.7e5, xmax=0.8e5
#ax2.set_ylim(ymin=-1.0e5, ymax=1.75e5)
ax2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax2.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax2.yaxis.get_major_formatter().set_powerlimits((0, 0))

#ax2.set_title('')
ax2.set_xlabel('x [pc]')
ax2.set_ylabel('z [pc]')

plt.show()

### Core evolution code vs1

In [ ]:
# Mean positions of galaxy form header
fid_cluster_mean_pos = fid_mod_header[2:5, 0]
print(np.shape(fid_cluster_mean_pos))

# Centering galaxy in coordinate system using header positions
fid_cluster_pos = fid_mod_data[:, 1:4, 0] - fid_cluster_mean_pos

# Calculating distance from the center to all particles in galaxy
fid_part_pos_in_cluster = np.sqrt(np.sum(fid_cluster_pos**2, axis=1))
print(np.shape(fid_part_pos_in_cluster))
print(np.min(fid_part_pos_in_cluster))
print(np.max(fid_part_pos_in_cluster))

# Which particles are inside certain radius of core
core_part_rows = np.where(fid_part_pos_in_cluster<=(2.17e-3*1))[0]

# Extracting core particles
fid_core_part = fid_cluster_pos[core_part_rows, :]
print(np.shape(fid_core_part))

In [ ]:
figt1 = plt.figure(figsize=(10, 8))
axt1 = figt1.add_subplot(projection='3d', computed_zorder=False)

#axt1.scatter(fid_cluster_pos[:, 0], fid_cluster_pos[:, 1], fid_cluster_pos[:, 2], s=5, c='b', alpha=0.05)
sc = axt1.scatter(fid_core_part[:, 0], fid_core_part[:, 1], fid_core_part[:, 2], s=10, c=fid_core_part[:, 1], 
             alpha=0.5, cmap='spring')

plt.colorbar(sc, label='y [kpc]')
axt1.set_xlabel('x [kpc]', labelpad=15)
axt1.set_ylabel('y [kpc]', labelpad=15)
axt1.set_zlabel('z [kpc]', labelpad=5)
axt1.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt1.xaxis.get_major_formatter().set_powerlimits((0, 0))
axt1.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt1.yaxis.get_major_formatter().set_powerlimits((0, 0))
axt1.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt1.zaxis.get_major_formatter().set_powerlimits((0, 0))

axt1.view_init(4, 45)


plt.show()

calculate distance between all core particle and extract the longest distance for every timestep and see how it evolves over time and see how/if the core is being disrupted and when. 

In [ ]:
fid_core_part_all = np.empty((len(core_part_rows), 4, 40))
mean_dists = np.empty((40))

for i in range(40):
    # Shape N, 3, 1
    tstep_pos = fid_mod_data[core_part_rows, 0:4, i]
    fid_core_part_all[:, :, i] = tstep_pos
    
    # calculate distance between all core patricles in x, y, and z
    diffs = tstep_pos[:, np.newaxis, 1:] - tstep_pos[np.newaxis, :, 1:]
    #print(diffs)
    #Calculates the r-distance between all core particles
    #This gives a double-sided square array divided by the diagonal, which is 0
    dists = np.linalg.norm(diffs, axis=2)
    #print(dists)
    #print()
    mean_dists[i] = np.mean(dists[np.triu_indices(10, k=1)])
    
    
    
print(np.shape(fid_core_part_all))
print(np.shape(mean_dists))
print(mean_dists)

In [ ]:
figt3 = plt.figure(figsize=(10, 8))
axt3 = figt3.add_subplot(projection='3d', computed_zorder=False)

#axt1.scatter(fid_cluster_pos[:, 0], fid_cluster_pos[:, 1], fid_cluster_pos[:, 2], s=5, c='b', alpha=0.05)
sc = axt3.scatter(fid_core_part_all[:, 1, -1], fid_core_part_all[:, 2, -1], fid_core_part_all[:, 3, -1], 
                  s=10, c=fid_core_part_all[:, 2, -1], alpha=0.5, cmap='spring')

plt.colorbar(sc, label='y [kpc]')
axt3.set_xlabel('x [kpc]', labelpad=20)
axt3.set_ylabel('y [kpc]', labelpad=20)
axt3.set_zlabel('z [kpc]', labelpad=10)
axt3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt3.xaxis.get_major_formatter().set_powerlimits((0, 0))
axt3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt3.yaxis.get_major_formatter().set_powerlimits((0, 0))
axt3.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axt3.zaxis.get_major_formatter().set_powerlimits((0, 0))

axt3.view_init(4, 45)


plt.show()

# Core evolution

### Core evolution analysis function

In [ ]:
def core_evolution(rmin, rmax, n_shells, in_pos, in_mean_pos, data):
    """
    Parameters:
    -----------
    rmin: int or float
        The minimum considered radius of the core
    
    rmax: int or float
        The maximum considered radius of the core
        
    n_shells: int
        Number of core shells to use to calculate evolution as a function of radius
    
    in_pos: array
        Initial positions of particles in galaxy
        
    in_mean_pos: array
        Initial mean position of galaxy (found in PETAR header)
        
    data: array
        All data for all particles in all snapshots
    ------------------------------------------------------------------------------------
    """
    
    # Centering galaxy in coordinate system using header positions
    centered_data = in_pos - in_mean_pos
    
    # Calculating distance from the center to all particles in galaxy
    r_parts = np.sqrt(np.sum(centered_data**2, axis=1))
    
    # Array of all core radii
    core_radii = np.linspace(rmin, rmax, n_shells)
    
    n_snapshots = len(data[0, 0, :])
    
    # Mean core distances for all cores for all snapshots, shape:(n_snapshots, core_radii)
    # Mean distance between particles in core
    mean_core_dists = np.empty((n_snapshots, len(core_radii)))
    # Mean position of the core for all snapshots, shape:(n_snapshots, 3), 3 = x, y, z
    core_mean_pos = np.empty((n_snapshots, 3))
    # shapes: (n_snapshots, n_cores)
    a_vals = np.empty((n_snapshots, len(core_radii)))
    b_vals = np.empty((n_snapshots, len(core_radii)))
    c_vals = np.empty((n_snapshots, len(core_radii)))
    
    # Loops over different core radii, calculations for all snapshots are done in one go
    for i, core_rad in enumerate(core_radii):      
        # Which particles are inside certain radius of core
        core_part_rows = np.where(r_parts<=(core_rad))[0]

        # Extracting core particles' data for all timesteps
        core_parts_all_steps = data[core_part_rows, :, :]
        # Shape (n_core_part, 3, n_snapshots) ##(N, 3, 1) now (N, 3, 40)
        core_pos = core_parts_all_steps[:, 1:4, :] 
        #print(f'{np.shape(core_pos) = }')
        
        
        
        # Calculating mean distances between core particles for all timesteps
        # ----------------------------------------------------------------------------------------------
        # Distance between all particles in core in all coordinates
        # Gives shape:(N_part, N_part, 3, n_snapshots)
        pos_diffs = core_pos[:, np.newaxis, :, :] - core_pos[np.newaxis, :, :, :]
        #print(f'{np.shape(pos_diffs) = }')
        
        # Calculates the r-distance between all core particles
        # This gives a double-sided square array divided by the diagonal, which is 0
        # Shape: (n_core_part, n_core_part, n_snapshots)
        pos_dists = np.linalg.norm(pos_diffs, axis=2)
        #print(f'{np.shape(pos_dists) = }')
        
        # Shape: (n_snapshots)
        pos_mean_dists = np.mean(pos_dists[np.triu_indices(len(core_pos), k=1)], axis=0)
        #print(f'{np.shape(pos_mean_dists) = }')
        
        mean_core_dists[:, i] = pos_mean_dists#[:, np.newaxis]
        
        
        
        # Mean core position over time
        # ----------------------------------------------------------------------------------------
        if i==0:
            core_mean_pos[:, :] = np.mean(core_pos, axis=0).T # Should get shape (N_snapshots, 3)
        
        
        
        # Inertia tensor and aspect ratios
        # ---------------------------------------------------------------------------------------------
        # Loops over the snapshots
        for j in range(n_snapshots):
            print(j)
            # Shape (n_core_part, 1)
            core_masses_tstep = core_parts_all_steps[:, 0, j]
            #print(core_masses)
            #print(np.shape(core_masses))
            #print(np.shape(core_pos[:, :, j]))
            #row, col = np.shape(core_pos[:, :, j])
            core_pos_tstep = core_pos[:, :, j] #np.reshape(core_pos[:, :, j], (row, col))
            #print(core_pos_tstep)
            print(np.max(core_pos_tstep[:, 0]) - np.min(core_pos_tstep[:, 0]))
            print(np.max(core_pos_tstep[:, 1]) - np.min(core_pos_tstep[:, 1]))
            print(np.max(core_pos_tstep[:, 2]) - np.min(core_pos_tstep[:, 2]))
            
            # Calculating inertia tensor using my function
            core_in_tensor = inertia_tensor(core_masses_tstep, core_pos_tstep)
            #print(core_in_tensor)
            
            # Diagonalising tensor and extract eigenvalues and eigenvectors
            core_eig_vals, core_eig_vec = np.linalg.eig(core_in_tensor)
            
            
            fig = plt.figure(figsize=(8, 8))
            ax = fig.add_subplot(projection='3d')

            ax.scatter(core_pos_tstep[:, 0], core_pos_tstep[:, 1], core_pos_tstep[:, 2], c='b', s=10)
            #lims = 10
            ##ax.set_xlim(-lims, lims)
            ##ax.set_ylim(-lims, lims)
            ##ax.set_zlim(-lims, lims)
            
            ax.set_xlabel('x', labelpad=20)
            ax.set_ylabel('y', labelpad=20)
            ax.set_zlabel('z', labelpad=20)

            ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
            ax.xaxis.get_major_formatter().set_powerlimits((0, 0))
            ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
            ax.yaxis.get_major_formatter().set_powerlimits((0, 0))
            ax.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
            ax.zaxis.get_major_formatter().set_powerlimits((0, 0))

            plt.show()
            
            # Assigning eigenvalues according to size
            # I1 <= I2 <= I3
            eig1 = np.min(core_eig_vals)
            eig3 = np.max(core_eig_vals)
            eig2_pos = np.where((core_eig_vals!=eig1)&(core_eig_vals!=eig3))[0]
            eig2 = core_eig_vals[eig2_pos][0]
            print(eig1, eig2, eig3)
            print()

            a_vals[j, i], b_vals[j, i], c_vals[j, i] = aspect_ratios(eig1, eig2, eig3, np.sum(core_masses_tstep))
            #print(a_vals[j, i])
            
    
    # Shape (n_snapshots, n_cores, 3)
    core_aspect_ratios = np.concatenate([a_vals[:, :, np.newaxis], b_vals[:, :, np.newaxis], c_vals[:, :, np.newaxis]], axis=2)
    
    return core_mean_pos, mean_core_dists, core_aspect_ratios

In [ ]:
n_shells = 5
fid_mean_core_pos, fid_core_mean_dists, fid_core_aspect_ratios = core_evolution(2.17e-3, 2.17e-3*n_shells, 
                                                                                n_shells, fid_mod_data[:, 1:4, 0], 
                                                                                fid_mod_header[2:5, 0], 
                                                                                fid_mod_data)

In [ ]:
print(np.shape(fid_mean_core_pos)) # should have shape:(40, 3)
print(np.shape(fid_core_mean_dists)) # should have shape:(40, 5)
print(np.shape(fid_core_aspect_ratios)) # should have shape:(40, 5, 3)
#print(fid_core_mean_dists)

In [ ]:
print(fid_core_aspect_ratios[:, 0, 0])
#print()
#print(fid_core_aspect_ratios[:, -1, 0])

In [ ]:
figt1, axt1 = plt.subplots(3, 1, figsize=(10, 17))

tsteps = np.arange(0, 40, 1)*128

rcs = np.round(np.linspace(1, 5, 5)*2.17, 2)

axt1[0].plot(tsteps, fid_core_aspect_ratios[:, 0, 0], color='darkblue', label=rf'$r_c = {rcs[0]} pc$')
axt1[0].plot(tsteps, fid_core_aspect_ratios[:, 1, 0], color='blue', label=rf'$r_c = {rcs[1]} pc$')
axt1[0].plot(tsteps, fid_core_aspect_ratios[:, 2, 0], color='c', label=rf'$r_c = {rcs[2]} pc$')
axt1[0].plot(tsteps, fid_core_aspect_ratios[:, 3, 0], color='lime', label=rf'$r_c = {rcs[3]} pc$')
axt1[0].plot(tsteps, fid_core_aspect_ratios[:, 4, 0], color='darkgreen', label=rf'$r_c = {rcs[4]} pc$')

axt1[0].set_xlabel('Time [Myr]')
axt1[0].set_ylabel('Aspect ratio [kpc?]')
axt1[0].set_title('a')
axt1[0].legend()



axt1[1].plot(tsteps, fid_core_aspect_ratios[:, 0, 1], color='darkblue', label=rf'$r_c = {rcs[0]} pc$')
axt1[1].plot(tsteps, fid_core_aspect_ratios[:, 1, 1], color='blue', label=rf'$r_c = {rcs[1]} pc$')
axt1[1].plot(tsteps, fid_core_aspect_ratios[:, 2, 1], color='c', label=rf'$r_c = {rcs[2]} pc$')
axt1[1].plot(tsteps, fid_core_aspect_ratios[:, 3, 1], color='lime', label=rf'$r_c = {rcs[3]} pc$')
axt1[1].plot(tsteps, fid_core_aspect_ratios[:, 4, 1], color='darkgreen', label=rf'$r_c = {rcs[4]} pc$')

axt1[1].set_xlabel('Time [Myr]')
axt1[1].set_ylabel('Aspect ratio [kpc?]')
axt1[1].set_title('b')
axt1[1].legend()



axt1[2].plot(tsteps, fid_core_aspect_ratios[:, 0, 2], color='darkblue', label=rf'$r_c = {rcs[0]} pc$')
axt1[2].plot(tsteps, fid_core_aspect_ratios[:, 1, 2], color='blue', label=rf'$r_c = {rcs[1]} pc$')
axt1[2].plot(tsteps, fid_core_aspect_ratios[:, 2, 2], color='c', label=rf'$r_c = {rcs[2]} pc$')
axt1[2].plot(tsteps, fid_core_aspect_ratios[:, 3, 2], color='lime', label=rf'$r_c = {rcs[3]} pc$')
axt1[2].plot(tsteps, fid_core_aspect_ratios[:, 4, 2], color='darkgreen', label=rf'$r_c = {rcs[4]} pc$')

axt1[2].set_xlabel('Time [Myr]')
axt1[2].set_ylabel('Aspect ratio [kpc?]')
axt1[2].set_title('c')
axt1[2].legend()


plt.tight_layout()
plt.savefig('./Plots/Aspect_ratios_fiducial_model.png', bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
figt1_5  = plt.figure(figsize=(8, 8))
axt1_5 = figt1_5.add_subplot(projection='3d')

axt1_5.plot(fid_mean_core_pos[:, 0], fid_mean_core_pos[:, 1], fid_mean_core_pos[:, 2], color='b')


#axt1_5.view_init(4, 45)

plt.show()

In [ ]:
fid_core_r = np.sqrt(np.sum(fid_mean_core_pos[:, :]**2, axis=1))

In [ ]:
figt1_52, axt1_52 = plt.subplots(figsize=(8, 8))

axt1_52.plot(tsteps, fid_core_r, color='b')


#axt1_52.view_init(4, 45)

plt.show()

### Extract core particle data in all timesteps

In [ ]:
figt2, axt2 = plt.subplots(figsize=(10, 8))



tsteps = np.arange(0, 40, 1)*128
print(np.shape(tsteps))

axt2.scatter(tsteps, fid_core_mean_dists[:, 0])

axt2.set_xlabel('Time [Myr]')
axt2.set_ylabel('Mean distance in core [kpc]')
axt2.set_title(rf'Mean distance in core over time: $r_c$={2.17}pc')

#plt.savefig('./Plots/Mean_core_distance_421part_over_time.png', facecolor='w')
plt.show()

### Core intertia calculations

### Core evolution animation

In [ ]:
len_data = len(fid_core_part_all)
f_anim_3d = np.arange(0, 40, 1)
    
#pos_in = np.round(data_header[2:5, 0], 3)
#vel_in = np.round(data_header[5:, 0], 3)


fig_anim_3d = plt.figure(figsize=(10, 10)) 
ax_anim_3d = fig_anim_3d.add_subplot(projection='3d')

scatter1 = ax_anim_3d.scatter([], [], [], c='b', s=10)

tx=-1.5e2
ty=2.0e2
tz=2.0e2

my_text = ax_anim_3d.text(tx, ty, tz, '', fontsize=20) #-30500, 35500, 35500


u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

x_bulge = 1.500 * np.outer(np.cos(u), np.sin(v))
y_bulge = 1.500 * np.outer(np.sin(u), np.sin(v))
z_bulge = 1.500 * np.outer(np.ones(np.size(u)), np.cos(v))

x_disc = 14.490 * np.outer(np.cos(u), np.sin(v))
y_disc = 14.490 * np.outer(np.sin(u), np.sin(v))
z_disc = 0.300 * np.outer(np.ones(np.size(u)), np.cos(v))


min_lim = -25.0e1
max_lim = 25.0e1

def init_anim_3d(): 
    #ax_anim_3d.set_title(f'N: {len_data}, Pos: {pos_in} pc,'+ '\n' + f' Vel: {vel_in} km/s', fontsize=20, y=0.85)
    ax_anim_3d.set_xlabel('x [kpc]', labelpad = 15)
    ax_anim_3d.set_ylabel('y [kpc]', labelpad = 15)
    ax_anim_3d.set_zlabel('z [kpc]', labelpad = 5) 
    
    ax_anim_3d.set_xlim(xmin=min_lim, xmax=max_lim)
    ax_anim_3d.set_ylim(ymin=min_lim, ymax=max_lim)
    ax_anim_3d.set_zlim(zmin=min_lim, zmax=max_lim)
    
    # Fixing the ticklabels
    ax_anim_3d.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax_anim_3d.xaxis.get_major_formatter().set_powerlimits((0, 0))
    ax_anim_3d.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax_anim_3d.yaxis.get_major_formatter().set_powerlimits((0, 0))
    ax_anim_3d.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax_anim_3d.zaxis.get_major_formatter().set_powerlimits((0, 0))
    
    # Plots Milky Way
    ax_anim_3d.plot_surface(x_bulge, y_bulge, z_bulge, color='grey', alpha=0.5)
    ax_anim_3d.plot_surface(x_disc, y_disc, z_disc, color='grey', alpha=0.5)
    
    ax_anim_3d.view_init(4, 45)
    ax_anim_3d.dist=12
    
    plt.tight_layout()
    plt.show()
    
    return scatter1, 

def update_anim_3d(frame):
    my_text.set_text(f't={frame*128} Myr')
    
    scatter1._offsets3d = (fid_core_part_all[:, 1, frame], fid_core_part_all[:, 2, frame], 
                           fid_core_part_all[:, 3, frame])
    
    return scatter1, 



animation_plots_3d = animation.FuncAnimation(fig_anim_3d, update_anim_3d, 
                                             frames=f_anim_3d, init_func=init_anim_3d)

    
    
writervideo = animation.FFMpegWriter(fps=6)
animation_plots_3d.save(f'./Animations/Anim_Fid_large_model_core_787part_3d.mp4', writer=writervideo)

# Stream alignment code

In [ ]:
stars1_mask = fid_aligned2[:, -2]<=3.47e2
stars2_mask = (fid_aligned2[:, -2]>3.47e2)&(fid_aligned2[:, -2]<=3.55e2)
stars3_mask = fid_aligned2[:, -2]>3.55e2

stream_data1 = fid_aligned2[stars1_mask, :]
stream_data2 = fid_aligned2[stars2_mask, :]
stream_data3 = fid_aligned2[stars3_mask, :]



orbit1_mask = orbital_data[:, -1]<=3.47e2
orbit2_mask = (orbital_data[:, -1]>3.47e2)&(orbital_data[:, -1]<=3.55e2)
orbit3_mask = orbital_data[:, -1]>3.55e2

orbit1 = orbital_data[orbit1_mask, -1]
orbit2 = orbital_data[orbit2_mask, -1]
orbit3 = orbital_data[orbit3_mask, -1]

fid_width1 = fid_width[orbit1_mask]
fid_width2 = fid_width[orbit2_mask]
fid_width3 = fid_width[orbit3_mask]

fid_part_dens1 = fid_part_dens[orbit1_mask]
fid_part_dens2 = fid_part_dens[orbit2_mask]
fid_part_dens3 = fid_part_dens[orbit3_mask]

In [ ]:
figf2, axf2 = plt.subplots(figsize=(15, 5))

axf2.scatter(orbital_data[:, -1], fid_width, c='b', s=10)
axf2.scatter(orbit1, fid_width1, c='b', s=10)
axf2.scatter(orbit2, fid_width2, c='r', s=10)
axf2.scatter(orbit3, fid_width3, c='orange', s=10)

axf2.axvline(3.47e2, linestyle='dashed', c='r')
axf2.axvline(3.55e2, linestyle='dashed', c='r')

axf2.set_title('Width along stream')
axf2.set_xlabel('Distance in stream [kpc]')
axf2.set_ylabel('Width [kpc]')

axf2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf2.xaxis.get_major_formatter().set_powerlimits((0, 0))
axf2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf2.yaxis.get_major_formatter().set_powerlimits((0, 0))

plt.show()

In [ ]:
figf3, axf3 = plt.subplots(figsize=(15, 5))

axf3.scatter(orbital_data[:, -1], fid_part_dens, c='b', s=10)
axf3.scatter(orbit1, fid_part_dens1, c='b', s=10)
axf3.scatter(orbit2, fid_part_dens2, c='r', s=10)
axf3.scatter(orbit3, fid_part_dens3, c='orange', s=10)

axf3.set_title('Particle density along stream')
axf3.set_xlabel('Distance in stream [kpc]')
axf3.set_ylabel('Particle density')

axf3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf3.xaxis.get_major_formatter().set_powerlimits((0, 0))
axf3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf3.yaxis.get_major_formatter().set_powerlimits((0, 0))

axf3.set_ylim(ymin=0, ymax=0.4e3)

plt.show()

In [ ]:
figf4 = plt.figure(figsize=(15, 6))
axf4_1 = figf4.add_subplot(1, 2, 1, projection='3d')

axf4_1.scatter(stream_data1[:, 0], stream_data1[:, 1], stream_data1[:, 2], label='1st bump', 
             s=1, c='b', alpha=0.25)
axf4_1.scatter(stream_data2[:, 0], stream_data2[:, 1], stream_data2[:, 2], label='Mid peak', 
             s=1, c='r', alpha=0.25)
axf4_1.scatter(stream_data3[:, 0], stream_data3[:, 1], stream_data3[:, 2], label='2nd bump', 
             s=1, c='orange', alpha=0.25)

axf4_1.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf4_1.xaxis.get_major_formatter().set_powerlimits((0, 0))
axf4_1.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf4_1.yaxis.get_major_formatter().set_powerlimits((0, 0))
axf4_1.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf4_1.zaxis.get_major_formatter().set_powerlimits((0, 0))

axf4_1.view_init(4, 45)

axf4_1.set_xlabel('x [kpc]', labelpad=15)
axf4_1.set_ylabel('y [kpc]', labelpad=15)
axf4_1.set_zlabel('z [kpc]', labelpad=5)



axf4_2 = figf4.add_subplot(1, 2, 2)

axf4_2.scatter(stream_data1[:, 2], stream_data1[:, 1], label='1st bump', 
             s=1, c='b', alpha=0.25)
axf4_2.scatter(stream_data2[:, 2], stream_data2[:, 1], label='Mid peak', 
             s=1, c='r', alpha=0.25)
axf4_2.scatter(stream_data3[:, 2], stream_data3[:, 1], label='2nd bump', 
             s=1, c='orange', alpha=0.25)

axf4_2.plot(orbital_data[:, 2], orbital_data[:, 1], color='k')

axf4_2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf4_2.xaxis.get_major_formatter().set_powerlimits((0, 0))
axf4_2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf4_2.yaxis.get_major_formatter().set_powerlimits((0, 0))


axf4_2.set_xlabel('z [kpc]')
axf4_2.set_ylabel('y [kpc]')


plt.show()

In [ ]:
orb_vel = np.sqrt(np.sum(orbital_data[:, 0:3]**2, axis=1))
stream_vel = np.sqrt(np.sum(fid_stream_data[:, 0:3]**2, axis=1))

In [ ]:
figf5 = plt.figure(figsize=(20, 6))
axf5_1 = figf5.add_subplot(1, 2, 1, projection='3d')

sc1 = axf5_1.scatter(fid_stream_data[:, 0], fid_stream_data[:, 1], fid_stream_data[:, 2], label='Stream', 
             s=1, c=stream_vel, cmap='spring', alpha=0.25)

cbar1 = plt.colorbar(sc1, ax=axf5_1)
cbar1.formatter.set_powerlimits((0, 0))

axf5_1.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf5_1.xaxis.get_major_formatter().set_powerlimits((0, 0))
axf5_1.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf5_1.yaxis.get_major_formatter().set_powerlimits((0, 0))
axf5_1.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf5_1.zaxis.get_major_formatter().set_powerlimits((0, 0))

axf5_1.view_init(4, 45)

axf5_1.set_xlabel('x [pc]', labelpad=15)
axf5_1.set_ylabel('y [pc]', labelpad=15)
axf5_1.set_zlabel('z [pc]', labelpad=5)



axf5_2 = figf5.add_subplot(1, 2, 2)

sc21 = axf5_2.scatter(fid_stream_data[:, 2], fid_stream_data[:, 1], s=1, c=stream_vel, cmap='spring', alpha=0.25)
cbar21 = plt.colorbar(sc21, ax=axf5_2)
cbar21.formatter.set_powerlimits((0, 0))

sc22 = axf5_2.scatter(orbital_data[:, 2], orbital_data[:, 1], c=orb_vel, cmap='winter', s=0.5)
cbar22 = plt.colorbar(sc22, ax=axf5_2)
cbar22.formatter.set_powerlimits((0, 0))

axf5_2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf5_2.xaxis.get_major_formatter().set_powerlimits((0, 0))
axf5_2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axf5_2.yaxis.get_major_formatter().set_powerlimits((0, 0))


axf5_2.set_xlabel('z [pc]')
axf5_2.set_ylabel('y [pc]')


plt.show()

# Comparing fast and slow fiducial run

In [ ]:
figfids2, axfids2 = plt.subplots(1, 2, figsize=(15, 6))

axfids2[0].scatter(fid_small_mod_data[:, 1, -1], fid_small_mod_data[:, 3, -1], c='b', alpha=0.5, s=3, zorder=20)

axfids2[0].set_xlabel('x [kpc]')
axfids2[0].set_ylabel('z [kpc]')
axfids2[0].set_title(r'$N_{fid, s}=112558$')

axfids2[0].set_xlim(xmin=-80, xmax=80)
axfids2[0].set_ylim(ymin=-0.85e2, ymax=1.75e2)

axfids2[0].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axfids2[0].xaxis.get_major_formatter().set_powerlimits((0, 0))
axfids2[0].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axfids2[0].yaxis.get_major_formatter().set_powerlimits((0, 0))

ellipse = mpl.patches.Ellipse((0, 0), width=30.000, height=0.300, color='k', alpha=1) # Disc (x, z) view 30000pc and 300pc
circle = mpl.patches.Circle((0, 0), 1.500, color='k', alpha=1) # Bulge (x, z) view 1500pc

axfids2[0].add_patch(circle)
axfids2[0].add_patch(ellipse)
axfids2[0].grid(zorder=-100)


axfids2[1].scatter(fid_mod_data[:, 1, -1], fid_mod_data[:, 3, -1], c='b', alpha=0.5, s=3, zorder=20)

axfids2[1].set_xlabel('x [kpc]')
axfids2[1].set_ylabel('z [kpc]')
axfids2[1].set_title(r'$N_{fid}=997967$')

axfids2[1].set_xlim(xmin=-80, xmax=80)
axfids2[1].set_ylim(ymin=-0.85e2, ymax=1.75e2)

axfids2[1].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axfids2[1].xaxis.get_major_formatter().set_powerlimits((0, 0))
axfids2[1].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
axfids2[1].yaxis.get_major_formatter().set_powerlimits((0, 0))

ellipse2 = mpl.patches.Ellipse((0, 0), width=30.000, height=0.300, color='k', alpha=1) # Disc (x, z) view 30000pc and 300pc
circle2 = mpl.patches.Circle((0, 0), 1.500, color='k', alpha=1) # Bulge (x, z) view 1500pc

axfids2[1].add_patch(circle2)
axfids2[1].add_patch(ellipse2)
axfids2[1].grid(zorder=-100)


plt.tight_layout()
plt.subplots_adjust(wspace=0.25)
plt.savefig('./Plots/Fiducial_runs_comparison.png', facecolor='w', bbox_inches='tight')
plt.show()

### Comparing to trace-back snapshots

In [ ]:
fig3 = plt.figure(figsize=(15, 15))
grid3 = mpl.gridspec.GridSpec(2, 2)

a = [0, 0, 1, 1]
b = [0, 1, 0, 1]


trace_back_snapshots = np.linspace(0, 112, 8, dtype=int)
f_anim_3d_2 = np.arange(0, 8, 1)

min_lim = -1.0e2
max_lim = 1.0e2

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

its = [0, 2, 5, 7]

for i in range(4):
    it = its[i]
    x_bulge = 1.500 * np.outer(np.cos(u), np.sin(v))
    y_bulge = 1.500 * np.outer(np.sin(u), np.sin(v))
    z_bulge = 1.500 * np.outer(np.ones(np.size(u)), np.cos(v))

    x_disc = 14.490 * np.outer(np.cos(u), np.sin(v))
    y_disc = 14.490 * np.outer(np.sin(u), np.sin(v))
    z_disc = 0.300 * np.outer(np.ones(np.size(u)), np.cos(v))



    ax3 = fig3.add_subplot(grid3[a[i], b[i]], projection='3d')
    ax3.view_init(4, 45)
    ax3.scatter(trace_back_data[:, 1, trace_back_snapshots[::-1][it]], 
                   trace_back_data[:, 2, trace_back_snapshots[::-1][it]], 
                   trace_back_data[:, 3, trace_back_snapshots[::-1][it]], 
                   label='Trace-back', c='b', s=20)
    ax3.scatter(fid_mod_data[:, 1, it], fid_mod_data[:, 2, it], fid_mod_data[:, 3, it], 
                label='Fiducial', c='r', s=1)


    ax3.set_xlabel('x [kpc]', labelpad=20)
    ax3.set_ylabel('y [kpc]', labelpad=20)
    ax3.set_zlabel('z [kpc]', labelpad=10)
    ax3.set_title(f't = {it*128} Myr', fontsize=20, y=0.85)
    ax3.set_xlim(xmin=min_lim, xmax=max_lim)
    ax3.set_ylim(ymin=min_lim, ymax=max_lim)
    ax3.set_zlim(zmin=min_lim, zmax=max_lim)
    ax3.plot_surface(x_bulge, y_bulge, z_bulge, color='grey', alpha=0.5)
    ax3.plot_surface(x_disc, y_disc, z_disc, color='grey', alpha=0.5)
    ax3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax3.xaxis.get_major_formatter().set_powerlimits((0, 0))
    ax3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax3.yaxis.get_major_formatter().set_powerlimits((0, 0))
    ax3.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax3.zaxis.get_major_formatter().set_powerlimits((0, 0))
    
    lgnd = ax3.legend(loc = (0.55, 0.35))
    lgnd.legendHandles[0]._sizes=[30]
    lgnd.legendHandles[1]._sizes=[30]
    lgnd.legendHandles[0].set_alpha(1)
    lgnd.legendHandles[1].set_alpha(1)
    

ax3.dist=15

plt.tight_layout()
plt.subplots_adjust(wspace=0.175, hspace=0.0)
#plt.savefig('./Plots/Fiducial_large_run_and_traceback_snapshots.png', bbox_inches='tight', facecolor='w')
plt.show()